## ANALISIS DE DATOS DE FILMAFFINITY

In [3]:
import pandas as pd
import sqlite3

# ruta del csv
ruta_csv = "./data/filmaffinity_dataset.csv"

# Cargar el csv en un dataframe
df = pd.read_csv(ruta_csv)

# Conexión SQLite
conexion = sqlite3.connect("filmaffinity.db")

# Guardar el DataFrame como tabla en SQLite
df.to_sql("filmaffinity_tb", conexion, if_exists="replace", index=False)

print("Base de datos SQLite creada y datos importados")

cursor = conexion.cursor()

Base de datos SQLite creada y datos importados


In [5]:
consulta = """
           SELECT *
           FROM filmaffinity_tb
           LIMIT 10;
           """
resultado = pd.read_sql_query(consulta, conexion)
resultado

,Unnamed: 0,Título,Año,País,Dirección,Reparto,Nota,Tipo filme,Género
0,0,'49-'17,1917,Estados Unidos,Ruth Ann Baldwin,"Joseph W. Girard, Leo Pierson, William Dyer, M...",--,Película,Western
1,1,"10,000 Years B.C. (C)",1916,Estados Unidos,Willis H. O'Brien,None,"5,1",Cortometraje,Comedia
2,2,1812,1912,Rusia,"Vasili Goncharov, Kai Hansen, Aleksandr Uralsky","Pavel Knorr, Vasili Goncharov, Aleksandra Gonc...",--,Película,Drama
3,3,20.000 leguas de viaje submarino (C),1907,Francia,Georges Méliès,Georges Méliès,"6,0",Cortometraje,Fantástico
4,4,A Bad Case (C),1909,Francia,Émile Cohl,None,"5,3",Cortometraje,Comedia
5,5,A Bath Tub Elopement (C),1916,Estados Unidos,Marcel Perez,Marcel Perez,--,Cortometraje,Comedia
6,6,A Beast at Bay (C),1912,Estados Unidos,D.W. Griffith,"Mary Pickford, Edwin August, Alfred Paget, Mae...","5,4",Cortometraje,Drama
7,7,A Busy Night (C),1916,Estados Unidos,Marcel Perez,"Marcel Perez, Nilde Baracchi, Tom Murray",--,Cortometraje,Comedia
8,8,A Calamitous Elopement (C),1908,Estados Unidos,D.W. Griffith,"Harry Solter, Linda Arvidson, Charles Inslee, ...","4,6",Cortometraje,Comedia
9,9,A Child of the Ghetto (C),1910,Estados Unidos,D.W. Griffith,"Dorothy West, Kate Bruce, Dell Henderson, Char...",--,Cortometraje,Drama


# 1. Limpieza de data
En esta face haremos lo siguiente:

 1. Hacer un copia de la base de datos original para luego poder trabajar en esta copia y no afectar a la original.
 2. Luego pasamos a hacer la limpieza que consiste en:
      
      ✅ Datos faltantes (Nulos).

      ✅ Registros duplicados.

      ✅ Formatos inconsistentes (fechas, nombres, números).
      
      ✅ Valores atípicos (outliers).

### Hacer una copia de la db original para trabajar con esta copia

In [ ]:
# Nombre de la nueva base de datos
copia_db = "./data/copia.db"

# Ejecutar SQL para hacer la copia
consulta = f"""
ATTACH DATABASE '{copia_db}' AS copia;
SELECT sql FROM sqlite_master WHERE type='table';
"""
cursor.executescript(consulta)

# Copiar cada tabla a la base de datos de respaldo
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

for tabla in tablas:
    nombre_tabla = tabla[0]
    consulta_copia = f"CREATE TABLE copia.{nombre_tabla} AS SELECT * FROM {nombre_tabla};"
    cursor.execute(consulta_copia)

print("Copia de seguridad creada con SQL.")

In [11]:
consulta = "SELECT * FROM copia.filmaffinity_tb LIMIT 5;"
df_copia = pd.read_sql_query(consulta, conexion)
df_copia

,Unnamed: 0,Título,Año,País,Dirección,Reparto,Nota,Tipo filme,Género
0,0,'49-'17,1917,Estados Unidos,Ruth Ann Baldwin,"Joseph W. Girard, Leo Pierson, William Dyer, M...",--,Película,Western
1,1,"10,000 Years B.C. (C)",1916,Estados Unidos,Willis H. O'Brien,None,"5,1",Cortometraje,Comedia
2,2,1812,1912,Rusia,"Vasili Goncharov, Kai Hansen, Aleksandr Uralsky","Pavel Knorr, Vasili Goncharov, Aleksandra Gonc...",--,Película,Drama
3,3,20.000 leguas de viaje submarino (C),1907,Francia,Georges Méliès,Georges Méliès,"6,0",Cortometraje,Fantástico
4,4,A Bad Case (C),1909,Francia,Émile Cohl,None,"5,3",Cortometraje,Comedia


In [40]:
# eliminar_tabla = "DROP TABLE filmaffinity_copia;"
# eliminar_tabla = pd.read_sql_query(eliminar_tabla, conexion)
# eliminar_tabla

In [12]:
consulta_renombrar = "ALTER TABLE copia.filmaffinity_tb RENAME TO filmaffinity_copia;"
conexion.execute(consulta_renombrar)
conexion.commit()  # Guardar los cambios
print("Tabla renombrada correctamente.")

Tabla renombrada correctamente.


### ✅ Revisar datos faltantes (Nulos)

In [13]:
revisar_nulos = """
                SELECT COUNT(*) AS nulos -- Para que no ingore los NULL
                FROM copia.filmaffinity_copia
                WHERE Nota LIKE "%--%" OR Nota IS NULL OR TRIM(Nota) = '';
                """
resultado = pd.read_sql_query(revisar_nulos, conexion)
resultado

# Reemplazar las inconsistencias de notas por 0
#reemplazar = """
#            SELECT REPLACE(Nota, "--", "Sin_Calificar") AS Nota_Actualizada FROM copia.filmaffinity_copia;
#            """
#resultado_2 = pd.read_sql_query(reemplazar, conexion)
#resultado_2

,nulos
0,50709


In [14]:
info_tabla = "PRAGMA table_info(filmaffinity_copia);"
info_tabla = pd.read_sql_query(info_tabla, conexion)
info_tabla

,cid,name,type,notnull,dflt_value,pk
0,0,Unnamed: 0,INT,0,None,0
1,1,Título,TEXT,0,None,0
2,2,Año,INT,0,None,0
3,3,País,TEXT,0,None,0
4,4,Dirección,TEXT,0,None,0
5,5,Reparto,TEXT,0,None,0
6,6,Nota,TEXT,0,None,0
7,7,Tipo filme,TEXT,0,None,0
8,8,Género,TEXT,0,None,0


In [15]:
revisar_nulos = """
                SELECT COUNT(*) AS nulos -- Para que no ingore los NULL
                FROM copia.filmaffinity_copia
                WHERE Nota LIKE "%--%" OR Nota IS NULL OR TRIM(Nota) = '';
                """
resultado = pd.read_sql_query(revisar_nulos, conexion)
resultado

,nulos
0,50709


In [16]:
actualizar = """
            UPDATE copia.filmaffinity_copia
            SET Nota = "Sin Calificar"
            WHERE Nota = "--"
            """
conexion.execute(actualizar)
conexion.commit()
print("Notas actualizadas correctamente")

Notas actualizadas correctamente


In [17]:
actualizar_n = """
               UPDATE copia.filmaffinity_copia
               SET Nota = REPLACE(Nota, ",", ".")
               WHERE Nota LIKE "%,%";
               """
conexion.execute(actualizar_n)
conexion.commit()
print("Valores Actualizados correctamente")

Valores Actualizados correctamente


In [18]:
# Valores Nulos en la columna Reparto
reparto_nulos = """
                SELECT COUNT(*) as total_nulos
                FROM copia.filmaffinity_copia
                WHERE Reparto IS NULL;
               """
reparto_nulos = pd.read_sql_query(reparto_nulos, conexion)
reparto_nulos # Y lo dejamos como NULL porque no afecta al análisis

,total_nulos
0,24478


In [27]:
total_nulos = """
              SELECT COUNT(*) AS total_nulos
              FROM copia.filmaffinity_copia
              WHERE Título IS NULL
                OR Año IS NULL
                OR País IS NULL
                OR Dirección IS NULL
                OR Reparto IS NULL
                OR Nota IS NULL
                OR `Tipo filme` IS NULL
                OR Género IS NULL
              """
total_nulos = pd.read_sql_query(total_nulos, conexion)
total_nulos

,total_nulos
0,24963


### ✅ Registros duplicados.

In [28]:
revisar_registros = """
                    SELECT COUNT(DISTINCT Título) AS distintos_registros
                    FROM copia.filmaffinity_copia;
                     """
revisar_registros= pd.read_sql_query(revisar_registros, conexion)
revisar_registros

,distintos_registros
0,111096


In [29]:
total_registros = """
                  SELECT COUNT(Título) AS registros_totales
                  FROM copia.filmaffinity_copia;
                  """
total_registros = pd.read_sql_query(total_registros, conexion)
total_registros

,registros_totales
0,119003


In [30]:
registros_totales = 119003
registros_unicos = 111096
total_duplicados = registros_totales - registros_unicos
total_duplicados

7907

In [31]:
identifar_duplicados = """
                    SELECT *,
                    ROW_NUMBER() OVER(PARTITION BY Título ORDER BY Título) AS num_fila
                    FROM copia.filmaffinity_copia;
                    """
identifar_duplicados = pd.read_sql_query(identifar_duplicados, conexion)
identifar_duplicados                    

,Unnamed: 0,Título,Año,País,Dirección,Reparto,Nota,Tipo filme,Género,num_fila
0,44167,Beasts of Prey,1985,Corea del Sur,Kim Ki-young,Kim Seong-gyeom,Sin Calificar,Película,Romance,1
1,22992,Dos contra Al Capone,1966,Italia,Giorgio Simonelli,"Franco Franchi, Ciccio Ingrassia, José Calvo, ...",Sin Calificar,Película,Comedia,1
2,91468,El Criptozoólogo (C),2015,España,Vicente Mallols,None,5.0,Cortometraje,"Animación, Aventuras, Fantástico",1
3,103892,El Criptozoólogo (C),2015,España,Vicente Mallols,None,5.0,Cortometraje,"Animación, Aventuras, Fantástico",2
4,44168,El retorno de Godzilla,1984,Japón,Koji Hashimoto,"Ken Tanaka, Yasuko Sawaguchi, Keiju Kobayashi,...",5.5,Película,"Ciencia ficción, Fantástico",1
...,...,...,...,...,...,...,...,...,...,...
118998,44164,‎Au cœur du cristal (C),1980,Francia,Gaël Badaud,Teo Hernández,Sin Calificar,Cortometraje,Documental,1
118999,44165,— ——– (Rock and Roll Movie) (C),1967,Estados Unidos,"Thom Andersen, Malcolm Brodwick",None,Sin Calificar,Cortometraje,Documental,1
119000,44166,‘36 to ‘77,1978,Reino Unido,"Marc Karlin, Jon Sanders, James Scott, Humphre...",None,Sin Calificar,Documental,Documental,1
119001,103891,‘71,2014,Reino Unido,Yann Demange,"Jack O'Connell, Paul Anderson, Valene Kane, Se...",6.6,Película,"Acción, Drama",1


In [32]:
duplicados = """
            SELECT *
            FROM (
                SELECT *,
                ROW_NUMBER() OVER(PARTITION BY Título ORDER BY Título) AS num_fila
                FROM copia.filmaffinity_copia
            ) subconsulta
            WHERE num_fila > 1;
             """
duplicados = pd.read_sql_query(duplicados, conexion)
duplicados

,Unnamed: 0,Título,Año,País,Dirección,Reparto,Nota,Tipo filme,Género,num_fila
0,103892,El Criptozoólogo (C),2015,España,Vicente Mallols,None,5.0,Cortometraje,"Animación, Aventuras, Fantástico",2
1,103894,#Horror,2015,Estados Unidos,Tara Subkoff,"Chloë Sevigny, Timothy Hutton, Balthazar Getty...",3.6,Película,Terror,2
2,103902,(Des)honestos,2015,Estados Unidos,Yael Melamede,Dan Ariely,6.6,Documental,Documental,2
3,91481,1,2013,Estados Unidos,Paul Crowder,"Niki Lauda, Mario Andretti, Michael Schumacher...",7.3,Documental,Documental,2
4,103912,10000 Years Later,2015,China,Yi Li,None,Sin Calificar,Película,Fantástico,2
...,...,...,...,...,...,...,...,...,...,...
7902,70981,Éxtasis,1995,España,Mariano Barroso,"Javier Bardem, Federico Luppi, Leire Berrocal,...",6.2,Película,Drama,2
7903,114225,Órfãos do Eldorado,2015,Brasil,Guilherme Coelho,"Milton Aires, Adriano Barroso, Henrique da Paz...",Sin Calificar,Película,Intriga,2
7904,91460,Última llamada,2002,Estados Unidos,Joel Schumacher,"Colin Farrell, Kiefer Sutherland, Forest Whita...",6.5,Película,"Acción, Thriller",2
7905,114226,Última oportunidad (Miniserie de TV),2015,Francia,François Velle,"Alexandra Lamy, Pascal Elbé, Lionel Abelanski,...",5.5,Miniserie,Thriller,2


In [ ]:
## Eliminamos duplicados
eliminar = """
            WITH duplicados AS (
                SELECT ROWID, ROW_NUMBER() OVER(PARTITION BY Título ORDER BY ROWID) AS num_fila
                FROM copia.filmaffinity_copia
            )
            DELETE FROM copia.filmaffinity_copia
            WHERE ROWID IN (
                SELECT ROWID FROM duplicados WHERE num_fila > 1
            );
           """
eliminar = pd.read_sql_query(eliminar, conexion)
eliminar

### ✅ Formatos inconsistentes (fechas, nombres, números).

In [34]:
# ✅ Inconsistencias en fechas:
# En este caso como es un año no hay problemas

# ✅ Inconsistencias en nombres
inco_nombres = """
                SELECT DISTINCT País FROM copia.filmaffinity_copia;
               """
inco_nombres = pd.read_sql_query(inco_nombres, conexion)
inco_nombres
# No hay inconsistencias en los nombres


# ✅ Inconsistencias en los números
# pues no hay inconsistencias ya que anteriormente actualizamos por ejemplo la "," por "."


,País
0,Estados Unidos
1,Rusia
2,Francia
3,Suecia
4,Dinamarca
...,...
181,Brunei
182,Islas Caimán
183,Ruanda
184,Kosovo


### ✅ Cambiar tipos de datos

El archivo copia.db-journal es un archivo temporal de journaling que crea SQLite cuando se ejecuta una transacción. Sirve para garantizar la integridad de la base de datos en caso de fallos o interrupciones inesperadas.

Si este archivo quedó atascado o la consulta no se ejecutó bien, puedes solucionarlo:

✅ SOLUCIÓN: Asegurar el COMMIT de la transacción
Si estás usando pd.read_sql_query(), este método solo se usa para consultas SELECT. Como UPDATE modifica datos, debes usar conexion.execute() y hacer commit() manualmente:

In [37]:
actualizar_d = """
    UPDATE copia.filmaffinity_copia
    SET Nota = CAST(Nota AS REAL)
    WHERE Nota NOT LIKE '%[^0-9.]%';
"""

conexion.execute(actualizar_d)  # Ejecutar la consulta
conexion.commit()  # Confirmar los cambios en la base de datos
print("Datos actualizados correctamente.")

Datos actualizados correctamente.


In [41]:
agregar_columna = "ALTER TABLE copia.filmaffinity_copia ADD COLUMN Nota_Nueva REAL";
conexion.execute(agregar_columna)
conexion.commit()
print("Nueva columna creada con éxito")

Nueva columna creada con éxito


In [43]:
# Copiar data de la columna Nota a Nota_nueva
copiar = """
        UPDATE copia.filmaffinity_copia
        SET Nota_Nueva = CAST(Nota AS REAL)
        WHERE Nota NOT LIKE '%[^0-9.]%'
         """
conexion.execute(copiar)
conexion.commit()
print("Datos copiados correctamente")

Datos copiados correctamente


In [ ]:
# Crear una nueva tabla con los valores actualizados
crear_tabla = """
            CREATE TABLE
              """

In [42]:
# Veamos los tipos de datos:
tipos_datos = "PRAGMA table_info(filmaffinity_copia);"
tipos_datos = pd.read_sql_query(tipos_datos, conexion)
tipos_datos

,cid,name,type,notnull,dflt_value,pk
0,0,Unnamed: 0,INT,0,None,0
1,1,Título,TEXT,0,None,0
2,2,Año,INT,0,None,0
3,3,País,TEXT,0,None,0
4,4,Dirección,TEXT,0,None,0
5,5,Reparto,TEXT,0,None,0
6,6,Nota,TEXT,0,None,0
7,7,Tipo filme,TEXT,0,None,0
8,8,Género,TEXT,0,None,0
9,9,Nota_Nueva,REAL,0,None,0


In [ ]:
# Cerrar conexión con
# conexion.close()